In [3]:
import os, moment
os.environ['ECHO_SECRETS_LOCATION'] = '/home/jjorissen/echo_secrets.conf'
from echo_api import Connection
connection = Connection()

### I am attempting to add a Contact Log to a Physician. 

From the API documentation I know the following about ContactLog:

| Screen Name        | Caption           | Database Table  | Reference | Update|
|-------------|--------------|--------------|---------|
|CallLog|Contact Log|ContactLog|PhysicianID|AUD|

According to the Documenation, AUD means that Adding, Updating, and Deleting is supported. Since 'T' is not in the update column, I believe that these entries are created using `API_UpdateData` like the [`EditPhysicianMedicalLicenses`](#EditPhysicianMedicalLicenses) function in the example API.

`connection.client.service.API_UpdateData(connection.session_id, treename, levelname, screenName, nameSpace, parameters, dsXML)`
   * `session` (string): the session id returned from a login.
   * `treename` (string) the name of tree (for providers, this is “Locations”)
   * `levelname` (string) the name of the level (for providers, this is “Providers”)
   * `screenName` (string) name of the screen.
   * `nameSpace` (string) the name space that the screen belongs to. Currently always “Symed”.
   * `parameters` (string) the parameters for the select statement. It is possible to get the names of the parameters from the API_SelectParameters function, the values will need to be supplied by the programmer.
   * `dsXML` (string) the XML data containing the updates.

From the above I know what `session`, `screenname`, and `nameSpace` should be, and I think I know what `treename`, `parameters`, and `dsXML` should be. I can only guess that `levelname` must be `Providers`, since the only thing I know about `levelnames` is shown above. So with this information in mind I tried the following:

I created a wrapper function called `add_contact_log_entry` to add a contact log entry to a Physician by ID. Each function call prints the relevant portion of the `dsXML` parameter (the actual schema contains the data definitions but the excerpt printed only contains what I'm trying to pass to the update. An example of the full schema can be found at the bottom of the document).

These are the examples I have of contact logs:

In [5]:
connection.show_physician_contact_log(306, show_all=True)

[SchemaInnerDict([('SECURITY_CONTEXT', None),
                  ('CallID', '4'),
                  ('ContactDate', '2017-09-06T00:00:00'),
                  ('FollowUpCompleted', 'false'),
                  ('ContactLogTypeID', '5'),
                  ('ContactID', '0'),
                  ('UserDisplayName', 'Kathleen Reynolds'),
                  ('ContactName', 'Test Test'),
                  ('EntityGuid', 'e43361b7-59f3-4e1f-92c1-1faa1f840b0f'),
                  ('Subject', 'Simple Contact Log')]),
 SchemaInnerDict([('CallID', '4'),
                  ('TimeEdited', 'AAAAAAAD/DY='),
                  ('Subject', 'Simple Contact Log')]),
 SchemaInnerDict([('SECURITY_CONTEXT', None),
                  ('CallID', '5'),
                  ('ContactDate', '2017-09-06T00:00:00'),
                  ('FollowUpDate', '2017-09-13T00:00:00'),
                  ('FollowUpCompleted', 'false'),
                  ('ContactLogTypeID', '5'),
                  ('Notes',
                   'This is a 

## So, based on the above I have tried:

### 1

Below is the equivalent to `API_UpdateData(session, 'Locations', 'Provider', 'CallLog', 'Symed', '@EntityGuid|60e04aba-f548-41c6-9194-4c5dde6d4359|guid', updated_schema1)`

Full [updated_schema1](#updated_schema1) at bottom of document.

In [6]:
connection.add_contact_log_entry(307, Notes="I am a new minimal note!")

<NewDataSet>
   <Table>
     <Notes>I am a new minimal note!</Notes>
    </Table>
  </NewDataSet>



'Error: Error:SymedData.UpdateDataSpecified cast is not valid.'

### 2

Below is the equivalent to `API_UpdateData(session, 'Locations', 'Provider', 'CallLog', 'Symed', '@EntityGuid|60e04aba-f548-41c6-9194-4c5dde6d4359|guid', updated_schema2)`

Full [updated_schema2](#updated_schema2) at bottom of document.

In [7]:
connection.add_contact_log_entry(307, Notes="I am a new minimal note!",
                                 UserDisplayName="Kathleen Reynolds", ContactName="JP", 
                                 Subject="Not a whole lot",
                                 ContactLogTypeID=5)

<NewDataSet>
   <Table>
     <Notes>I am a new minimal note!</Notes>
      <UserDisplayName>Kathleen Reynolds</UserDisplayName>
      <ContactName>JP</ContactName>
      <Subject>Not a whole lot</Subject>
      <ContactLogTypeID>5</ContactLogTypeID>
    </Table>
  </NewDataSet>



'Error: Error:SymedData.UpdateDataSpecified cast is not valid.'

### 3

Below is the equivalent to `API_UpdateData(session, 'Locations', 'Provider', 'CallLog', 'Symed', '@EntityGuid|60e04aba-f548-41c6-9194-4c5dde6d4359|guid', updated_schema3)`

Full [updated_schema3](#updated_schema3) at bottom of document.

In [4]:
connection.add_contact_log_entry(307, CallID=6, Notes="I am a new minimal note!", ContactID=0, 
                                 UserDisplayName="Kathleen Reynolds", ContactName="JP", 
                                 Subject="Not a whole lot",
                                 ContactDate=moment.utcnow().strftime("%Y-%m-%dT%H:%M:%S"),
                                 ContactLogTypeID=5)

<NewDataSet>
   <Table>
     <CallID>6</CallID>
      <Notes>I am a new minimal note!</Notes>
      <ContactID>0</ContactID>
      <UserDisplayName>Kathleen Reynolds</UserDisplayName>
      <ContactName>JP</ContactName>
      <Subject>Not a whole lot</Subject>
      <ContactDate>2017-09-06T18:54:37</ContactDate>
      <ContactLogTypeID>5</ContactLogTypeID>
    </Table>
  </NewDataSet>



'Error: Error:SymedData.UpdateDataSpecified cast is not valid.'

I've tried several other configurations but I think this is enough for demonstration. I don't know what else to try.

# Resources

## EditPhysicianMedicalLicenses
<a id='EditPhysicianMedicalLicenses'></a>

In [ ]:
 public string EditPhysicianMedicalLicenses(string physicianid)
{
    string result = "";
    try
    {
        string parms = "@PhysicianID|" + physicianid + "|int";

        //we get the data here
        result = service.API_GetData(session, "MedicalLicenses", "Symed", parms);

        //********** start of modifications ******************
        DataSet dataSet = new DataSet();
        dataSet.Locale = CultureInfo.InvariantCulture;
        using (StringReader stringReader = new StringReader(result))
        {
            dataSet.ReadXml(stringReader, XmlReadMode.ReadSchema);
        }

        dataSet.AcceptChanges();

        DataTable dt = dataSet.Tables[0];
        if (dt.Rows.Count > 0)
        {
            //we'll edit the first one
            DataRow dr = dt.Rows[0];
            dr["LicenseNumber"] = "API123";
            dr["LicenseStatus"] = "Edited by API";
        }
        if (dt.Rows.Count > 1)
        {
            //we'll delete the second one
            DataRow dr = dt.Rows[1];
            dr.Delete();
        }
        DataRow drn = dt.NewRow();
        //have to set the foriegn key when adding rows
        drn["PhysicianID"] = Convert.ToInt32(physicianid);
        drn["LicenseNumber"] = "API1";
        drn["LicenseStatus"] = "Added by API";
        dt.Rows.Add(drn);

        drn = dt.NewRow();
        //have to set the foriegn key when adding rows
        drn["PhysicianID"] = Convert.ToInt32(physicianid);
        drn["LicenseNumber"] = "API2";
        drn["LicenseStatus"] = "Added by API";
        dt.Rows.Add(drn);

        drn = dt.NewRow();
        //have to set the foriegn key when adding rows
        drn["PhysicianID"] = Convert.ToInt32(physicianid);
        drn["LicenseNumber"] = "API3";
        drn["LicenseStatus"] = "Added by API";
        dt.Rows.Add(drn);

        dt.AcceptChanges();

        XElement serializedDataSet = new XElement("SchemaandData");
        serializedDataSet.Add(XElement.Parse(dataSet.GetXmlSchema()));
        serializedDataSet.Add(XElement.Parse(dataSet.GetXml()));
        //********** end of modifications ******************

        //we update the data here
        result = service.API_UpdateData(session, "Locations", "Provider", "MedicalLicenses", "Symed", parms, serializedDataSet.ToString());

        service.API_Logout(session);
    }
    catch (Exception ex)
    {
        //getting here means the call failed without getting to the service
        result = ex.Message;
    }
    return result;

}

## updated_schema1 
<a id="updated_schema1"></a>

    <SchemaandData xmlns:ns1="urn:schemas-microsoft-com:xml-msdata" xmlns:xs="http://www.w3.org/2001/XMLSchema">
     <xs:schema id="NewDataSet">
       <xs:element name="NewDataSet" ns1:IsDataSet="true" ns1:UseCurrentLocale="true">
         <xs:complexType>
           <xs:choice maxOccurs="unbounded" minOccurs="0">
             <xs:element name="Table">
               <xs:complexType>
                 <xs:sequence>
                   <xs:element minOccurs="0" name="SECURITY_CONTEXT" type="xs:string" />
                    <xs:element minOccurs="0" name="CallID" type="xs:int" />
                    <xs:element minOccurs="0" name="ContactDate" type="xs:dateTime" ns1:DateTimeMode="Unspecified" />
                    <xs:element minOccurs="0" name="FollowUpDate" type="xs:dateTime" ns1:DateTimeMode="Unspecified" />
                    <xs:element minOccurs="0" name="FollowUpCompleted" type="xs:boolean" />
                    <xs:element minOccurs="0" name="ContactLogTypeID" type="xs:int" />
                    <xs:element minOccurs="0" name="Notes" type="xs:string" />
                    <xs:element minOccurs="0" name="ContactID" type="xs:int" />
                    <xs:element minOccurs="0" name="Notified" type="xs:int" />
                    <xs:element minOccurs="0" name="UserDisplayName" type="xs:string" />
                    <xs:element minOccurs="0" name="EMail" type="xs:string" />
                    <xs:element minOccurs="0" name="Phone" type="xs:string" />
                    <xs:element minOccurs="0" name="ContactName" type="xs:string" />
                    <xs:element minOccurs="0" name="Attachment" type="xs:string" />
                    <xs:element minOccurs="0" name="TrackingGuid" type="xs:string" ns1:DataType="System.Guid, mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089" />
                    <xs:element minOccurs="0" name="EntityGuid" type="xs:string" ns1:DataType="System.Guid, mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089" />
                    <xs:element minOccurs="0" name="Subject" type="xs:string" />
                    <xs:element minOccurs="0" name="grid_sort" type="xs:string" />
                  </xs:sequence>
                </xs:complexType>
              </xs:element>
              <xs:element name="Table1">
               <xs:complexType>
                 <xs:sequence>
                   <xs:element minOccurs="0" name="CallID" type="xs:int" />
                    <xs:element minOccurs="0" name="TimeEdited" type="xs:base64Binary" />
                    <xs:element minOccurs="0" name="Subject" type="xs:string" />
                  </xs:sequence>
                </xs:complexType>
              </xs:element>
            </xs:choice>
          </xs:complexType>
        </xs:element>
      </xs:schema>
      <NewDataSet>
       <Table>
         <Notes>I am a new minimal note!</Notes>
        </Table>
      </NewDataSet>
    </SchemaandData>

## updated_schema2 
<a id="updated_schema2"></a>

    <SchemaandData xmlns:ns1="urn:schemas-microsoft-com:xml-msdata" xmlns:xs="http://www.w3.org/2001/XMLSchema">
     <xs:schema id="NewDataSet">
       <xs:element name="NewDataSet" ns1:IsDataSet="true" ns1:UseCurrentLocale="true">
         <xs:complexType>
           <xs:choice maxOccurs="unbounded" minOccurs="0">
             <xs:element name="Table">
               <xs:complexType>
                 <xs:sequence>
                   <xs:element minOccurs="0" name="SECURITY_CONTEXT" type="xs:string" />
                    <xs:element minOccurs="0" name="CallID" type="xs:int" />
                    <xs:element minOccurs="0" name="ContactDate" type="xs:dateTime" ns1:DateTimeMode="Unspecified" />
                    <xs:element minOccurs="0" name="FollowUpDate" type="xs:dateTime" ns1:DateTimeMode="Unspecified" />
                    <xs:element minOccurs="0" name="FollowUpCompleted" type="xs:boolean" />
                    <xs:element minOccurs="0" name="ContactLogTypeID" type="xs:int" />
                    <xs:element minOccurs="0" name="Notes" type="xs:string" />
                    <xs:element minOccurs="0" name="ContactID" type="xs:int" />
                    <xs:element minOccurs="0" name="Notified" type="xs:int" />
                    <xs:element minOccurs="0" name="UserDisplayName" type="xs:string" />
                    <xs:element minOccurs="0" name="EMail" type="xs:string" />
                    <xs:element minOccurs="0" name="Phone" type="xs:string" />
                    <xs:element minOccurs="0" name="ContactName" type="xs:string" />
                    <xs:element minOccurs="0" name="Attachment" type="xs:string" />
                    <xs:element minOccurs="0" name="TrackingGuid" type="xs:string" ns1:DataType="System.Guid, mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089" />
                    <xs:element minOccurs="0" name="EntityGuid" type="xs:string" ns1:DataType="System.Guid, mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089" />
                    <xs:element minOccurs="0" name="Subject" type="xs:string" />
                    <xs:element minOccurs="0" name="grid_sort" type="xs:string" />
                  </xs:sequence>
                </xs:complexType>
              </xs:element>
              <xs:element name="Table1">
               <xs:complexType>
                 <xs:sequence>
                   <xs:element minOccurs="0" name="CallID" type="xs:int" />
                    <xs:element minOccurs="0" name="TimeEdited" type="xs:base64Binary" />
                    <xs:element minOccurs="0" name="Subject" type="xs:string" />
                  </xs:sequence>
                </xs:complexType>
              </xs:element>
            </xs:choice>
          </xs:complexType>
        </xs:element>
      </xs:schema>
      <NewDataSet>
       <Table>
         <Notes>I am a new minimal note!</Notes>
          <UserDisplayName>Kathleen Reynolds</UserDisplayName>
          <ContactName>JP</ContactName>
          <Subject>Not a whole lot</Subject>
          <ContactLogTypeID>5</ContactLogTypeID>
        </Table>
      </NewDataSet>
    </SchemaandData>

## updated_schema3 
<a id="updated_schema3"></a>

    <SchemaandData xmlns:ns1="urn:schemas-microsoft-com:xml-msdata" xmlns:xs="http://www.w3.org/2001/XMLSchema">
     <xs:schema id="NewDataSet">
       <xs:element name="NewDataSet" ns1:IsDataSet="true" ns1:UseCurrentLocale="true">
         <xs:complexType>
           <xs:choice maxOccurs="unbounded" minOccurs="0">
             <xs:element name="Table">
               <xs:complexType>
                 <xs:sequence>
                   <xs:element minOccurs="0" name="SECURITY_CONTEXT" type="xs:string" />
                    <xs:element minOccurs="0" name="CallID" type="xs:int" />
                    <xs:element minOccurs="0" name="ContactDate" type="xs:dateTime" ns1:DateTimeMode="Unspecified" />
                    <xs:element minOccurs="0" name="FollowUpDate" type="xs:dateTime" ns1:DateTimeMode="Unspecified" />
                    <xs:element minOccurs="0" name="FollowUpCompleted" type="xs:boolean" />
                    <xs:element minOccurs="0" name="ContactLogTypeID" type="xs:int" />
                    <xs:element minOccurs="0" name="Notes" type="xs:string" />
                    <xs:element minOccurs="0" name="ContactID" type="xs:int" />
                    <xs:element minOccurs="0" name="Notified" type="xs:int" />
                    <xs:element minOccurs="0" name="UserDisplayName" type="xs:string" />
                    <xs:element minOccurs="0" name="EMail" type="xs:string" />
                    <xs:element minOccurs="0" name="Phone" type="xs:string" />
                    <xs:element minOccurs="0" name="ContactName" type="xs:string" />
                    <xs:element minOccurs="0" name="Attachment" type="xs:string" />
                    <xs:element minOccurs="0" name="TrackingGuid" type="xs:string" ns1:DataType="System.Guid, mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089" />
                    <xs:element minOccurs="0" name="EntityGuid" type="xs:string" ns1:DataType="System.Guid, mscorlib, Version=4.0.0.0, Culture=neutral, PublicKeyToken=b77a5c561934e089" />
                    <xs:element minOccurs="0" name="Subject" type="xs:string" />
                    <xs:element minOccurs="0" name="grid_sort" type="xs:string" />
                  </xs:sequence>
                </xs:complexType>
              </xs:element>
              <xs:element name="Table1">
               <xs:complexType>
                 <xs:sequence>
                   <xs:element minOccurs="0" name="CallID" type="xs:int" />
                    <xs:element minOccurs="0" name="TimeEdited" type="xs:base64Binary" />
                    <xs:element minOccurs="0" name="Subject" type="xs:string" />
                  </xs:sequence>
                </xs:complexType>
              </xs:element>
            </xs:choice>
          </xs:complexType>
        </xs:element>
      </xs:schema>
      <NewDataSet>
       <Table>
         <CallID>6</CallID>
          <Notes>I am a new minimal note!</Notes>
          <ContactID>0</ContactID>
          <UserDisplayName>Kathleen Reynolds</UserDisplayName>
          <ContactName>JP</ContactName>
          <Subject>Not a whole lot</Subject>
          <ContactDate>2017-09-06T19:25:09</ContactDate>
          <ContactLogTypeID>5</ContactLogTypeID>
        </Table>
      </NewDataSet>
    </SchemaandData>